# ETL Data Warehouse Tutorial
## Building a Complete BI System for Retail Analytics

This notebook teaches you how to:
1. **Extract** raw data from CSV files
2. **Transform** data into a dimensional model
3. **Load** data into a Data Warehouse
4. **Query** the data warehouse for business insights
5. **Calculate** KPIs for decision making

---

## Part 1: Understanding Dimensional Modeling

### OLTP vs OLAP
- **OLTP** (Online Transaction Processing): Normalized databases for fast transactions (your RAW data)
- **OLAP** (Online Analytical Processing): Denormalized data warehouse for fast queries (your DW)

### Star Schema
A star schema consists of:
- **Fact Table**: Contains metrics (measures) and foreign keys to dimensions
  - Example: Sales Fact Table with quantity, revenue, cost, profit
- **Dimension Tables**: Contain descriptive attributes
  - Product Dimension (category, brand, price)
  - Customer Dimension (city, country, age)
  - Channel Dimension (store type)
  - Date Dimension (day, month, quarter, year)

### Example: Star Schema for Retail
```
                    DIM_DATE
                      |
        DIM_PRODUCT - FACT_SALES - DIM_CUSTOMER
                      |
                   DIM_CHANNEL
```

## Part 2: Setup - Import Libraries

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set up paths
project_root = r'c:\Users\LENOVO\OneDrive\Documentos\ETL\3 Lab\ETL-lab-3'
raw_data_path = os.path.join(project_root, 'data', 'raw')
warehouse_path = os.path.join(project_root, 'data', 'warehouse')
etl_path = os.path.join(project_root, 'ETL')
db_path = os.path.join(warehouse_path, 'datawarehouse.db')

# Create warehouse folder if it doesn't exist
os.makedirs(warehouse_path, exist_ok=True)

# Add ETL folder to Python path
sys.path.insert(0, etl_path)

print("✓ Environment configured")
print(f"  Raw data path: {raw_data_path}")
print(f"  Database path: {db_path}")

## Part 3: EXTRACT Phase

The Extract phase reads raw data from CSV files and validates the schema.

**Key Concepts:**
- Read files in a structured way
- Validate that all expected columns exist
- Check for empty tables
- Convert data types appropriately

In [ ]:
# Import the extract module
from extract import DataExtractor

# Initialize the extractor
extractor = DataExtractor(raw_data_path)

# Extract all data
extracted_data = extractor.extract_all()

# Show what was extracted
print("\nExtracted Data Summary:")
for table_name, df in extracted_data.items():
    print(f"\n{table_name.upper()}:")
    print(f"  Shape: {df.shape}")
    print(f"  Columns: {list(df.columns)}")

In [ ]:
# Preview the raw data - Products
print("Products Table:")
print(extracted_data['products'].head())

In [ ]:
print("\nSales Table:")
print(extracted_data['sales'].head())

In [ ]:
print("\nCustomers Table:")
print(extracted_data['customers'].head())

In [ ]:
print("\nChannels Table:")
print(extracted_data['channels'])

## Part 4: TRANSFORM Phase

The Transform phase prepares data for the data warehouse.

**What happens here:**
1. **Create Date Dimension** - Useful for time-based analysis (trends, seasonality)
2. **Create Product Dimension** - Denormalized product data with calculations
3. **Create Customer Dimension** - Customer attributes
4. **Create Channel Dimension** - Sales channel information
5. **Create Fact Table** - Transactions with measures (quantity, revenue, profit)

**Key Concepts:**
- **Surrogate Keys**: Sequential integer IDs (better for DW than business keys)
- **Measures**: Numeric values that can be aggregated (sum, avg)
- **Attributes**: Descriptive fields used for filtering and grouping

In [ ]:
# Import the transform module
from transform import DataTransformer

# Initialize the transformer
transformer = DataTransformer(extracted_data)

# Transform all data
transformed_data = transformer.transform_all()

print("\nTransformed Data Summary:")
for table_name, df in transformed_data.items():
    print(f"\n{table_name.upper()}:")
    print(f"  Shape: {df.shape}")

In [ ]:
# Let's examine the Date Dimension
print("DATE DIMENSION (First few rows):")
print(transformed_data['dim_date'].head(10))
print(f"\nDate range: {transformed_data['dim_date']['date'].min()} to {transformed_data['dim_date']['date'].max()}")

In [ ]:
# Examine Product Dimension with calculated margins
print("PRODUCT DIMENSION (with calculated margins):")
product_check = transformed_data['dim_product'][['product_key', 'product_id', 'name', 'category', 'brand', 'unit_price', 'unit_cost', 'margin_percent']]
print(product_check.head(10))
print(f"\nTotal products: {len(transformed_data['dim_product'])}")
print(f"Categories: {transformed_data['dim_product']['category'].unique()}")
print(f"Brands: {transformed_data['dim_product']['brand'].nunique()} brands")

In [ ]:
# Examine Sales Fact Table
print("SALES FACT TABLE (with calculated measures):")
print(transformed_data['fact_sales'].head())
print(f"\nFact table shape: {transformed_data['fact_sales'].shape}")
print(f"Total sales amount: ${transformed_data['fact_sales']['total_sales_amount'].sum():,.2f}")
print(f"Total profit: ${transformed_data['fact_sales']['profit'].sum():,.2f}")
print(f"Average profit margin: {transformed_data['fact_sales']['profit_margin'].mean():.2f}%")

## Part 5: LOAD Phase

The Load phase creates the Data Warehouse database and loads the dimensions and fact table.

**Loading Order:**
1. Create the schema (all tables)
2. Load dimension tables FIRST (no dependencies)
3. Load fact table LAST (uses foreign keys to dimensions)

**Key Concepts:**
- **Primary Key**: Unique identifier for each row
- **Foreign Key**: Reference to another table's primary key
- **Referential Integrity**: Ensures all foreign key references point to valid records

In [ ]:
# Import the load module
from load import DataWarehouseLoader

# Initialize the loader
loader = DataWarehouseLoader(db_path)

# Load all data into the warehouse
loader.load_all(transformed_data)

## Part 6: Query the Data Warehouse

Now we can query the warehouse to get business insights. The dimensional model makes these queries fast!

In [ ]:
# Connect to the data warehouse
conn = sqlite3.connect(db_path)

# Query 1: Total sales and profit by product category
query1 = """
SELECT 
    p.category,
    COUNT(f.sales_key) as num_transactions,
    SUM(f.quantity) as total_quantity,
    ROUND(SUM(f.total_sales_amount), 2) as total_revenue,
    ROUND(SUM(f.profit), 2) as total_profit,
    ROUND(AVG(f.profit_margin), 2) as avg_profit_margin
FROM fact_sales f
JOIN dim_product p ON f.product_key = p.product_key
GROUP BY p.category
ORDER BY total_revenue DESC
"""

kpi1 = pd.read_sql_query(query1, conn)
print("KPI 1: Sales Volume and Revenue by Product Category")
print(kpi1)

In [ ]:
# Query 2: Revenue by channel (Physical Store vs Online)
query2 = """
SELECT 
    c.channel,
    COUNT(f.sales_key) as num_transactions,
    SUM(f.quantity) as total_quantity,
    ROUND(SUM(f.total_sales_amount), 2) as total_revenue,
    ROUND(SUM(f.profit), 2) as total_profit,
    ROUND(100 * SUM(f.total_sales_amount) / (SELECT SUM(total_sales_amount) FROM fact_sales), 2) as revenue_pct
FROM fact_sales f
JOIN dim_channel c ON f.channel_key = c.channel_key
GROUP BY c.channel
ORDER BY total_revenue DESC
"""

kpi2 = pd.read_sql_query(query2, conn)
print("\nKPI 2: Revenue by Sales Channel")
print(kpi2)

In [ ]:
# Query 3: Monthly sales trends
query3 = """
SELECT 
    d.year,
    d.month,
    d.month_name,
    COUNT(f.sales_key) as num_transactions,
    ROUND(SUM(f.total_sales_amount), 2) as monthly_revenue,
    ROUND(SUM(f.profit), 2) as monthly_profit,
    ROUND(AVG(f.total_sales_amount), 2) as avg_transaction_value
FROM fact_sales f
JOIN dim_date d ON f.date_id = d.date_id
GROUP BY d.year, d.month, d.month_name
ORDER BY d.year, d.month
"""

kpi3 = pd.read_sql_query(query3, conn)
print("\nKPI 3: Monthly Sales Trends")
print(kpi3)

In [ ]:
# Query 4: Top brands by revenue
query4 = """
SELECT 
    p.brand,
    COUNT(f.sales_key) as num_transactions,
    SUM(f.quantity) as total_units_sold,
    ROUND(SUM(f.total_sales_amount), 2) as brand_revenue,
    ROUND(SUM(f.profit), 2) as brand_profit,
    ROUND(AVG(p.margin_percent), 2) as avg_margin_percent
FROM fact_sales f
JOIN dim_product p ON f.product_key = p.product_key
GROUP BY p.brand
ORDER BY brand_revenue DESC
"""

kpi4 = pd.read_sql_query(query4, conn)
print("\nKPI 4: Performance by Brand")
print(kpi4)

In [ ]:
# Query 5: Customer geography analysis
query5 = """
SELECT 
    c.country,
    COUNT(DISTINCT c.customer_key) as num_customers,
    COUNT(f.sales_key) as total_transactions,
    ROUND(SUM(f.total_sales_amount), 2) as country_revenue,
    ROUND(AVG(f.total_sales_amount), 2) as avg_transaction_value
FROM fact_sales f
JOIN dim_customer c ON f.customer_key = c.customer_key
GROUP BY c.country
ORDER BY country_revenue DESC
"""

kpi5 = pd.read_sql_query(query5, conn)
print("\nKPI 5: Sales by Customer Geography")
print(kpi5)

## Part 7: Data Warehouse Visualizations

Now let's create visual representations of our KPIs for business presentations.

In [ ]:
# Set up visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 10)

# Create a figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Data Warehouse KPI Dashboard', fontsize=16, fontweight='bold')

# Plot 1: Revenue by Category (Bar Chart)
ax1 = axes[0, 0]
kpi1_sorted = kpi1.sort_values('total_revenue', ascending=True)
ax1.barh(kpi1_sorted['category'], kpi1_sorted['total_revenue'], color='steelblue')
ax1.set_xlabel('Revenue ($)', fontweight='bold')
ax1.set_title('Revenue by Product Category', fontweight='bold')
ax1.grid(axis='x', alpha=0.3)
for i, v in enumerate(kpi1_sorted['total_revenue']):
    ax1.text(v + 5000, i, f'${v:,.0f}', va='center')

# Plot 2: Revenue by Channel (Pie Chart)
ax2 = axes[0, 1]
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
ax2.pie(kpi2['total_revenue'], labels=kpi2['channel'], autopct='%1.1f%%', 
        colors=colors, startangle=90, textprops={'fontsize': 10})
ax2.set_title('Revenue Distribution by Channel', fontweight='bold')

# Plot 3: Monthly Revenue Trend (Line Chart)
ax3 = axes[1, 0]
kpi3_sorted = kpi3.sort_values('month')
ax3.plot(kpi3_sorted['month_name'], kpi3_sorted['monthly_revenue'], 
         marker='o', linewidth=2, markersize=8, color='darkgreen')
ax3.fill_between(range(len(kpi3_sorted)), kpi3_sorted['monthly_revenue'], alpha=0.3, color='green')
ax3.set_xlabel('Month', fontweight='bold')
ax3.set_ylabel('Revenue ($)', fontweight='bold')
ax3.set_title('Monthly Revenue Trend', fontweight='bold')
ax3.grid(True, alpha=0.3)
ax3.tick_params(axis='x', rotation=45)

# Plot 4: Top Brands by Revenue (Horizontal Bar)
ax4 = axes[1, 1]
kpi4_top = kpi4.head(8).sort_values('brand_revenue', ascending=True)
ax4.barh(kpi4_top['brand'], kpi4_top['brand_revenue'], color='coral')
ax4.set_xlabel('Revenue ($)', fontweight='bold')
ax4.set_title('Top 8 Brands by Revenue', fontweight='bold')
ax4.grid(axis='x', alpha=0.3)
for i, v in enumerate(kpi4_top['brand_revenue']):
    ax4.text(v + 5000, i, f'${v:,.0f}', va='center')

plt.tight_layout()
plt.show()

print("\n✓ Dashboard created successfully!")

## Part 8: Advanced Analysis - Profitability Insights

In [ ]:
# Query: Product profitability analysis
query_products = """
SELECT 
    p.product_id,
    p.name,
    p.brand,
    p.category,
    COUNT(f.sales_key) as times_sold,
    ROUND(SUM(f.total_sales_amount), 2) as product_revenue,
    ROUND(SUM(f.profit), 2) as product_profit,
    ROUND(AVG(f.profit_margin), 2) as profit_margin,
    ROUND(SUM(f.total_sales_amount) - SUM(f.total_cost), 2) as total_profit
FROM fact_sales f
JOIN dim_product p ON f.product_key = p.product_key
GROUP BY p.product_id, p.name, p.brand, p.category
HAVING times_sold > 0
ORDER BY product_profit DESC
LIMIT 10
"""

top_products = pd.read_sql_query(query_products, conn)
print("Top 10 Most Profitable Products:")
print(top_products.to_string())

In [ ]:
# Query: Best and worst performing products
query_worst = """
SELECT 
    p.name,
    p.brand,
    COUNT(f.sales_key) as times_sold,
    ROUND(SUM(f.total_sales_amount), 2) as revenue,
    ROUND(SUM(f.profit), 2) as profit
FROM fact_sales f
JOIN dim_product p ON f.product_key = p.product_key
GROUP BY p.product_id, p.name, p.brand
HAVING times_sold > 0
ORDER BY profit ASC
LIMIT 5
"""

worst_products = pd.read_sql_query(query_worst, conn)
print("\nWorst 5 Performing Products (by profit):")
print(worst_products.to_string())

## Part 9: Business Summary & Key Insights

In [ ]:
# Calculate key metrics
summary_query = """
SELECT 
    COUNT(DISTINCT f.sales_key) as total_transactions,
    COUNT(DISTINCT f.customer_key) as unique_customers,
    COUNT(DISTINCT f.product_key) as products_sold,
    COUNT(DISTINCT f.date_id) as days_active,
    COUNT(DISTINCT f.channel_key) as channels,
    SUM(f.quantity) as total_units,
    ROUND(SUM(f.total_sales_amount), 2) as total_revenue,
    ROUND(SUM(f.total_cost), 2) as total_cost,
    ROUND(SUM(f.profit), 2) as total_profit,
    ROUND(100 * SUM(f.profit) / SUM(f.total_sales_amount), 2) as profit_margin_percent
FROM fact_sales f
"""

summary = pd.read_sql_query(summary_query, conn)

print("\n" + "="*60)
print("BUSINESS SUMMARY - KEY METRICS")
print("="*60)
print(f"Total Transactions: {summary['total_transactions'].values[0]:,}")
print(f"Unique Customers: {summary['unique_customers'].values[0]:,}")
print(f"Products Sold: {summary['products_sold'].values[0]:,}")
print(f"Active Days: {summary['days_active'].values[0]:,}")
print(f"Sales Channels: {summary['channels'].values[0]:,}")
print(f"Total Units Sold: {summary['total_units'].values[0]:,}")
print(f"\nTotal Revenue: ${summary['total_revenue'].values[0]:,.2f}")
print(f"Total Cost: ${summary['total_cost'].values[0]:,.2f}")
print(f"Total Profit: ${summary['total_profit'].values[0]:,.2f}")
print(f"Overall Profit Margin: {summary['profit_margin_percent'].values[0]:.2f}%")
print("="*60)

## Part 10: Running the Complete ETL Pipeline

Here's how to run the entire ETL process end-to-end:

In [ ]:
# Main ETL Pipeline
print("\n" + "="*60)
print("COMPLETE ETL PIPELINE EXECUTION")
print("="*60)

# STEP 1: EXTRACT
print("\nSTEP 1: EXTRACT")
print("-" * 60)
from extract import extract
extracted = extract(raw_data_path)

# STEP 2: TRANSFORM
print("\nSTEP 2: TRANSFORM")
print("-" * 60)
from transform import transform
transformed = transform(extracted)

# STEP 3: LOAD
print("\nSTEP 3: LOAD")
print("-" * 60)
from load import load
load(transformed, db_path)

print("\n" + "="*60)
print("✓ ETL PIPELINE COMPLETE!")
print("="*60)
print("\nYour Data Warehouse is ready for analysis!")

In [ ]:
# Close database connection
conn.close()
print("\n✓ Database connection closed")

## Learning Summary

### What You Learned:
1. **Dimensional Modeling** - Designing star schemas for analytics
2. **ETL Concepts** - Extract, Transform, Load stages
3. **Data Quality** - Schema validation and referential integrity
4. **Surrogate Keys** - Why and how to use them
5. **KPI Calculation** - Computing business intelligence metrics
6. **Visualization** - Creating dashboards from dimensional data

### Key Files Created:
- `extract.py` - Reads and validates raw data
- `transform.py` - Creates dimensions and fact tables
- `load.py` - Loads data into SQLite data warehouse
- `datawarehouse.db` - Your complete data warehouse

### Next Steps:
- Modify queries to answer specific business questions
- Add more dimensions (e.g., product reviews, customer segments)
- Create additional fact tables for different business processes
- Build dashboards in Power BI or Tableau
- Automate the ETL pipeline to run on schedules

### Data Warehouse Benefits:
✓ Fast queries for business analysis
✓ Centralized source of truth
✓ Easy to understand structure
✓ Supports complex reporting needs
✓ Foundation for ML/AI projects